In [ ]:
!pip install python-dotenv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import getpass
import os
import openai
from dotenv import load_dotenv


dotenv_path = '/content/drive/MyDrive/DATA298B/.env'
load_dotenv(dotenv_path)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')



In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install -q langchain langchain_community sentence_transformers modelscope torch datasets peft trl bitsandbytes accelerate transformers chromadb trulens trulens-apps-langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.8 MB/s eta 0

In [ ]:
import random
import re

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from peft import PeftModel
from peft import AutoPeftModelForCausalLM
from transformers import pipeline
#from langchain.llms import HuggingFacePipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def split_dialogue_Q_A(conversation):
    pattern = re.compile(r'Q: (.*?)\nA: (.*?)\n', re.DOTALL)
    matches = pattern.findall(conversation)
    questionlist=[]
    answerlist=[]

    for match in matches:
        question, answer = match

        # if len(question.split(' '))>50 or len(answer.split(' '))>50 or len(question.split(' '))<3:
        #     continue
        questionlist.append(question)
        answerlist.append(answer)

        # print("Question:", question)
        # print("Answer:", answer)
        # print("-----")
        # exit(1)

    return questionlist,answerlist

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "bigscience/bloom-7b1"
DEFAULT_SYSTEM_PROMPT = """
You are physicist Richard Feynman, well-known for your significant contributions to particle physics, quantum electrodynamics, and quantum mechanics. You have a ability for making difficult ideas understandable to others without diminishing their depth. You are known for your insatiable curiosity, sense of humor, and unorthodox teaching style. You enjoy solving problems and have a talent for telling stories.
""".strip()

In [ ]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(
        embeddings.model_name, device=DEVICE)

<ipython-input-9-75405089cb32>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
ragdb = Chroma(persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman", embedding_function=embeddings)

retriever = ragdb.as_retriever(search_kwargs={'k': 3})

<ipython-input-10-1e142866e725>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  ragdb = Chroma(persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman", embedding_function=embeddings)


In [ ]:
def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("A:")[-1]
    output += doc
    output += "\n\n"
  return output



def create_model_and_tokenizer(MODEL_NAME):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""
    ### Instruction: {system_prompt}
    {custom_format(retriever.invoke(question))}
    ### Question:
    {question.strip()}
    ### Response:[/INST]
    {response}
    """.strip()
    return tempt

def generate_testing_prompt(question, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""
    ### Instruction: {system_prompt}
    {custom_format(retriever.invoke(question))}
    ### Question:
    {question.strip()}
    ### Response:[/INST]
    """.strip()
    return tempt

def generate_response(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

def custom_format(docs):
  output = ""
  for doc in docs:
    #print(doc)
    doc = doc.page_content.split("A:")[-1]
    output += doc
    output += "\n"
  return output

In [ ]:
from transformers import AutoProcessor, AutoModelForPreTraining, BitsAndBytesConfig, AutoTokenizer
def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("A:")[-1]
    output += doc
    output += "\n"
  return output

def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""
    ### Instruction: {system_prompt}
    {custom_format(retriever.invoke(question))}
    ### Question:
    {question.strip()}
    ### Response:[/INST]
    {response}
    """.strip()
    return tempt

def create_model_and_processor(MODEL_NAME):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForPreTraining.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    processor = AutoProcessor.from_pretrained(MODEL_NAME)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, processor, tokenizer

MODEL_NAME = "bigscience/bloom-7b1"
model, processor, tokenizer = create_model_and_processor(MODEL_NAME)


config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
file_path = '/content/drive/MyDrive/DATA298B/data/interview_transcript_Q&A.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    texts = file.read()
questionlist,answerlist = split_dialogue_Q_A(texts)

tmpList = []
trainList = []
testList = []
for j,(question,answer) in enumerate(zip(questionlist,answerlist)):
  if type(question) != str:
    continue
  tmp_dict = {"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT)}
  # print(generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT))
  # exit(1)
  tmpList.append(tmp_dict)
  # if j == 200:
  #   break
  if j % 10== 0:
    print(j)
  if random.uniform(0,1) < .2:
    testList.append(tmp_dict)
  else:
    trainList.append(tmp_dict)

data = Dataset.from_list(tmpList)
train_data = Dataset.from_list(trainList)
test_data = Dataset.from_list(testList)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


In [ ]:
ragdb = Chroma(persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman", embedding_function=embeddings)
retriever = ragdb.as_retriever(search_kwargs={'k': 1})

In [ ]:
#OUTPUT_DIR = "trained_model4/checkpoint-90"
OUTPUT_DIR = "/content/drive/MyDrive/DATA298B/trained_model_feynman/5epoch/checkpoint-55"

newmodel = PeftModel.from_pretrained(model, OUTPUT_DIR)

text_generation_pipeline = pipeline(
    model=newmodel,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.6,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=100,
    top_k=30,
    top_p= 0.8,
    no_repeat_ngram_size =2
)
bloom_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [ ]:
prompt_template = """
    ### Instruction: You are an expert at impersonating animated characters. In particular, you are a world-leading expert at answering questions in the manner of physicst Richard Feynman.
[INST]Generate a response that sounds as close to what Richard Feynman would say. You can also use this auxiliary knowledge to help:
    - You are well-known for your significant contributions to particle physics, quantum electrodynamics, and quantum mechanics.
    - You have a ability for making difficult ideas understandable to others without diminishing their depth.
    - You are known for your insatiable curiosity, sense of humor, and unorthodox teaching style.
    - You enjoy solving problems and have a talent for telling stories.

    {context}
    ### Question:
    {question}
    ### Response:[/INST]
    """


prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=bloom_llm, prompt=prompt)

<ipython-input-16-885c1311fa20>:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=bloom_llm, prompt=prompt)


In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
chat_history=[]

def chatbot_response(message):
    global chat_history,rag_chain
    op = rag_chain.invoke(message)
    response = str(op['text']).split(':[/INST]')[-1]
    chat_history = chat_history + [{"role": "user", "content": message}, {"role": "assistant", "content": response}]
    return chat_history

In [ ]:
pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Textbox(placeholder="Enter your question"),
    ],
    outputs=[
        gr.Chatbot(type="messages",value=[])
    ],
    title="Chatbot-test",
    description="This is a test of the chatbot built using Gradio."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://66ceae8a2f7bb40b72.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install -q trulens trulens-apps-langchain trulens-providers-openai

In [ ]:
from trulens_eval import TruChain, Tru

tru = Tru()
tru.reset_database()

<ipython-input-22-1319694e3fb0>:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import TruChain, Tru


🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]
Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [ ]:
from trulens.providers.openai import OpenAI
from trulens_eval import Feedback
import numpy as np
from trulens.apps.langchain import TruChain
from ipytree import Tree, Node

provider = OpenAI()
context = TruChain.select_context(rag_chain)


# Groundedness
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons)
    .on(context.collect())
    .on_output()
    .on_input()
)

# Relevance between the overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

f_coherence = Feedback(provider.coherence_with_cot_reasons).on_output()

f_conciseness = Feedback(provider.conciseness_with_cot_reasons).on_output()

f_helpfulness = Feedback(provider.helpfulness_with_cot_reasons).on_output()

f_correctness = Feedback(provider.correctness_with_cot_reasons).on_output()

# Maliciousness
f_maliciousness = Feedback(provider.maliciousness).on_output()

# Harmfulness
f_harmfulness = Feedback(provider.harmfulness).on_output()

# Controversiality
f_controversiality = Feedback(provider.controversiality).on_output()

# Misogyny
f_misogyny = Feedback(provider.misogyny).on_output()

# Criminality
f_criminality = Feedback(provider.criminality).on_output()

# Insensitivity
f_insensitivity = Feedback(provider.insensitivity).on_output()



✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps__.context.invoke.rets[:].page_content.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In groundedness_measure_with_cot_reasons, input criteria will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps__.context.invoke.rets[:].page_content .
✅ In coherence_with_cot_reasons, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In conciseness_with_cot_reasons, input text will 

In [ ]:
tru_recorder = TruChain(rag_chain,
    app_id='BloomChain_feynman',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness, f_coherence, f_conciseness, f_helpfulness, f_correctness, f_maliciousness, f_harmfulness, f_controversiality, f_criminality, f_insensitivity, f_misogyny])

response, tru_record = tru_recorder.with_record(rag_chain.invoke, "Who is your best friend?")

json_like = tru_record.layout_calls_as_app()

In [ ]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))

    calls_node = Node('Calls')
    tree.add_node(calls_node)

    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)

        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)

    return tree

# Usage
tree = display_call_stack(json_like)

In [ ]:
json_like

Munch({'record_id': 'record_hash_df520310f25e9d436b48f5ee87854a93', 'app_id': 'app_hash_96c5c42a72ab48d58502250fb2d215da', 'cost': {'n_requests': 0, 'n_successful_requests': 0, 'n_completion_requests': 0, 'n_classification_requests': 0, 'n_classes': 0, 'n_embedding_requests': 0, 'n_embeddings': 0, 'n_tokens': 0, 'n_stream_chunks': 0, 'n_prompt_tokens': 0, 'n_completion_tokens': 0, 'n_cortex_guardrails_tokens': 0, 'cost': 0.0, 'cost_currency': 'USD'}, 'perf': {'start_time': '2024-12-02T00:17:13.064591', 'end_time': '2024-12-02T00:17:19.094666'}, 'ts': '2024-12-02T00:17:19.095188', 'tags': '-', 'meta': None, 'main_input': 'Who is your best friend?', 'main_output': "[{'id': None, 'metadata': {}, 'page_content': 'Q: By this time, your friend wasn’t with you in high school?\\nA: Yes, he was. Leonard. Not Bernard Walker, but Leonard Mortner had come from Cedarhurst and had moved to Far Rockaway, fortunately, and was with me in high school.', 'type': 'Document'}]", 'main_error': None, 'calls'

In [ ]:
counter = 0
with tru_recorder as recording:
    for x in testList:
        test_question = x['question']
        rag_chain.invoke(test_question)
        counter +=1
        if counter > 50:
            break


recs = recording.records
display(recs)

/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to proc

[Record(record_id='record_hash_f6d212ce97e9672a48ea88bd3acc12d4', app_id='app_hash_96c5c42a72ab48d58502250fb2d215da', cost=Cost(n_requests=0, n_successful_requests=0, n_completion_requests=0, n_classification_requests=0, n_classes=0, n_embedding_requests=0, n_embeddings=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, n_cortex_guardrails_tokens=0, cost=0.0, cost_currency='USD'), perf=Perf(start_time=datetime.datetime(2024, 12, 2, 0, 19, 18, 849579), end_time=datetime.datetime(2024, 12, 2, 0, 19, 24, 578564)), ts=datetime.datetime(2024, 12, 2, 0, 19, 24, 579102), tags='-', meta=None, main_input='This is a tape-recorded interview with Professor Richard Feynman, in his home in Altadena, California, on March 4, 1966. Charles Weiner will occasionally interrupt with questions. Let’s start at the beginning.', main_output="[{'id': None, 'metadata': {}, 'page_content': 'Q: This is a tape-recorded interview with Professor Richard Feynman, in his home in Altadena, Calif

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=["app_hash_96c5c42a72ab48d58502250fb2d215da"])

#records.head()

In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,conciseness_with_cot_reasons,relevance,maliciousness,harmfulness,criminality,controversiality,groundedness_measure_with_cot_reasons,context_relevance_with_cot_reasons,helpfulness_with_cot_reasons,insensitivity,correctness_with_cot_reasons,misogyny,coherence_with_cot_reasons
0,You were married in 1940 — something?,"[{'id': None, 'metadata': {}, 'page_content': 'Q: You were married in 1940 — something?\nA: 1941 or 1942, yeah. Details we’ll put in when the time comes, if you want. Then, when I was at MIT — I’ve cut the end of the high school. Yeah, that was the best girl, the most important girl. There were a few dates here and there with other girls, but this was the most important one.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,In high school.,"[{'id': None, 'metadata': {}, 'page_content': 'Q: It was past high school?\nA: In high school, somewhere, I probably stopped.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Then, with decent Regents’ scores and these applications —","[{'id': None, 'metadata': {}, 'page_content': 'Q: Then, with decent Regents’ scores and these applications —\nA: Fairly decent. Only fair. But good in the math and science.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,How did you do? We’ll look at the records too.,"[{'id': None, 'metadata': {}, 'page_content': 'Q: Do you remember how you did?\nA: Yes, I have all the records, if you want.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Excuse me a minute. Did you have, at that time, the state Regents’ examination?","[{'id': None, 'metadata': {}, 'page_content': 'Q: Excuse me a minute. Did you have, at that time, the state Regents’ examination?\nA: Yes, we did, on all the courses.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Well, it was also the Depression period. This would have emphasized it.","[{'id': None, 'metadata': {}, 'page_content': 'Q: Well, it was also the Depression period. This would have emphasized it.\nA: Yes. And also, to do some work like that, that’s good. I don’t know why, but that was the feeling — we should do it. I’m sure that if I didn’t, I would have got the spending money, because it wasn’t a lot of money, really. But the idea that you should get a job was somehow right. I don’t know (women’s tales) yeah…', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,What did you do there?,"[{'id': None, 'metadata': {}, 'page_content': 'Q: It was past high school?\nA: In high school, somewhere, I probably stopped.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Talking about these jobs — any other kind of job, more conventional in nature, in your high school period?","[{'id': None, 'metadata': {}, 'page_content': 'Q: Talking about these jobs — any other kind of job, more conventional in nature, in your high school period?\nA: Yeah, I had this job with a printer.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Two things occur to me here. Were you still in the house with the cousins at this time?,"[{'id': None, 'metadata': {}, 'page_content': 'Q: Two things occur to me here. Were you still in the house with the cousins at this time?\nA: No, no.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,So there’s no consistent poring through scientific libraries.,"[{'id': None, 'metadata': {}, 'page_content': 'Q: So there’s no consistent poring through scientific libraries.\nA: No, no. There was nowhere to find it. That I knew of. I think it’s just as well. I mean, I suspect — I know now — this has nothing to do with my history, but I suspect, I kind of try to imagine what would have happened to me if I’d lived in today’s era. I’m rather horrified. I think there are too many books, that the mind gets boggled. If I got interested, I would have so many things to look a

In [ ]:
records.iloc[9:13][["input", "output"] + feedback]

,input,output,conciseness_with_cot_reasons,relevance,maliciousness,harmfulness,criminality,controversiality,groundedness_measure_with_cot_reasons,context_relevance_with_cot_reasons,helpfulness_with_cot_reasons,insensitivity,correctness_with_cot_reasons,misogyny,coherence_with_cot_reasons
9,So there’s no consistent poring through scientific libraries.,"[{'id': None, 'metadata': {}, 'page_content': 'Q: So there’s no consistent poring through scientific libraries.\nA: No, no. There was nowhere to find it. That I knew of. I think it’s just as well. I mean, I suspect — I know now — this has nothing to do with my history, but I suspect, I kind of try to imagine what would have happened to me if I’d lived in today’s era. I’m rather horrified. I think there are too many books, that the mind gets boggled. If I got interested, I would have so many things to look at, I would go crazy. It’s too easy. Maybe. Maybe not. Maybe this is just an old fashioned point of view. You know. There’s always these things. But it does bother me a little bit that there’s so many things, and it’s so easy, and they’re watering down. The thing that I loved was, everything that I read was serious, was absolutely — wasn’t written for a child. I have never read anything that was written for a child except THE BOY SCIENTIST book, which was pretty good. But the things written for children, after a certain stage, by let’s say high school stage or even the late, late era in grammar school — things written for a child were not good. Very rarely, things written for a child — there were books on the cave man, you know? Cave men living, and people who lived on trees, and there was a book with a series of families in different circumstances that I found interesting. That was obvious for children. But later on, children’s things — I didn’t like children’s things. Because, for one thing I was very very — (and still am) — sensitive and very worried about was that the thing to be dead honest; that it isn’t fixed up so it looks easy. It isn’t fixed up and partly faked so that the explanation can be made more simply — for the child, you know? Details purposely left out, or slightly erroneous explanations, in order to get away with it. This was intolerable. So I ultimately had only to trust the completely mature and odd old things, even though I was only in high school. I think most kids in high school are very mature. Because I know my friends who were in literature — you know, who wrote — and my friends who wrote plays, and they would read the great plays. They wouldn’t read the children’s plays, you know. It’s the same thing. Anybody that’s any good in high school already knows that they had to look at the real stuff.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,You thought he should have grown the crystals.,"[{'id': None, 'metadata': {}, 'page_content': 'Q: You thought he should have grown the crystals.\nA: If he didn’t, he didn’t — if he’d grown small crystals, then these were nice, this is the way it is. You see, the question is “What is it?” not “How wonderful is it!” You don’t have to build it up. You don’t have to fake it. It’s great the way it is, is the sort of feeling. This is some kind of falseness, a terrible falseness. It just bothered me. But I just remember that when I’m thinking of — that they did have fairs, because I did see this there. I didn’t enter into that kind of thing. I didn’t make demonstrations for other students to watch, except in my own little club. And I didn’t have any feeling that I should enter into some fairs and things like that.', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"When was the next time that you dealt with it, when you had an opportunity to reflect on it again and use it?","[{'id': None, 'metadata': {}, 'page_content': 'Q: Stopped when?\nA: I don’t remember', 'type': 'Document'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,"How did that come about, t

In [ ]:
tru.get_leaderboard(app_ids=["app_hash_96c5c42a72ab48d58502250fb2d215da"])

,,coherence_with_cot_reasons,conciseness_with_cot_reasons,context_relevance_with_cot_reasons,controversiality,correctness_with_cot_reasons,criminality,groundedness_measure_with_cot_reasons,harmfulness,helpfulness_with_cot_reasons,insensitivity,maliciousness,misogyny,relevance,latency,total_cost
app_name,app_version,,,,,,,,,,,,,,,
BloomChain_feynman,base,0.409091,0.69697,0.757576,0.031746,0.439394,0.0,0.979798,0.0,0.409091,0.0,0.0,0.015873,0.545455,5.874684,0.0


In [ ]:
tru.run_dashboard()

<ipython-input-32-07ddb523643a>:1: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()


Starting dashboard ...
npm warn exec The following package was not found and will be installed: localtunnel@2.0.2

Go to this url and submit the ip given here. your url is: https://lemon-animals-divide.loca.lt

  Submit this IP Address: 34.87.166.46


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>